In [ ]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
})

In [ ]:
# Download data and create seurat objects
source("scTools/R/process.geo.data.r")
obj <- process.GEO.data(GEOID = "GSE264408")

In [1]:
obj <- get(load("objects/GSE264408.rda"))
obj

In [ ]:
# Merge data
merge_obj <- merge(x = obj[[1]],  y = c(obj[[2]], obj[[3]], obj[[4]], obj[[5]], obj[[6]], obj[[7]], obj[[8]], obj[[9]], obj[[10]]))

In [ ]:
obj <- JoinLayers(merge_obj)

In [ ]:
# Update metadata
obj@meta.data$Sample <- obj@meta.data$title
obj@meta.data$Treatment <- obj@meta.data$treatment.ch1
obj@meta.data$Disease <- obj@meta.data$disease.state.ch1

columns_to_keep <-  c('orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Sample', 'Treatment', 'Disease')
obj@meta.data <- obj@meta.data[, columns_to_keep, drop = FALSE]

In [ ]:
# Normalize, cluster, and visualize
options(future.globals.maxSize = 6000 * 1024^3)
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = FALSE)%>%
    RunPCA(npcs = 30, verbose = TRUE)%>%
    RunUMAP(reduction = "pca", dims = 1:20)%>%
    FindNeighbors(reduction = "pca", dims = 1:20)%>%
    FindClusters(resolution = 0.5)
save(obj, file = "objects/01.rda")

In [ ]:
# Load annotation
monaco.ref <- celldex::MonacoImmuneData()
blueprint.ref <- celldex::BlueprintEncodeData()

# Blueprint annnotation
sce <- LayerData(obj)
blueprint.main <- SingleR(test = sce, assay.type.test = 1, ref = blueprint.ref, labels = blueprint.ref$label.main)
obj@meta.data$blueprint.main <- blueprint.main$pruned.labels

# Monaco annontation
sce <- LayerData(obj)
monaco.fine <- SingleR(test = sce, assay.type.test = 1, ref = monaco.ref, labels = monaco.ref$label.fine)
obj@meta.data$monaco.fine <- monaco.fine$pruned.labels

In [ ]:
seu =  obj
scConf = createConfig(seu)
makeShinyApp(seu, scConf,
             gene.mapping = TRUE,
             shiny.title = "Colitis mouse colon scRNAseq (GSE264408)",
             shiny.dir = "colitis_scRNAseq_GSE264408/",
             gex.assay = "RNA",
             default.multigene = c('LILRB1', 'PDCD1')) 
system("R -e \"shiny::runApp('asthma_scRNAseq_GSE181710')\"")